In [5]:
import pandas as pd
import numpy as np
import pymssql
import warnings
warnings.filterwarnings("ignore") 

In [6]:
def conect_totvs():
    serv = '192.168.0.236'
    databa = 'PROTHEUS_PRODUCAO'
    user = 'ismael.silva'
    passa = 'w!1zayeUAM'
    conn = pymssql.connect(serv, user, passa, databa)
    cursor = conn.cursor(as_dict=True)
    return conn, cursor

In [15]:
con_tot, cursor = conect_totvs()
query = """SELECT 
    dbo.SPC010.PC_FILIAL AS Filial, 
    dbo.SPC010.PC_MAT AS Matricula, 
    dbo.SRA010.RA_NOME AS Funcionário, 
    dbo.SRA010.RA_CODFUNC AS [Cód Função], 
    dbo.SRJ010.RJ_DESC AS [Descrição Função], 
    dbo.SRA010.RA_CC AS [Cód CC], 
    dbo.CTT010.CTT_DESC01 AS [Descrição Centro de Custo], 
    dbo.SRA010.RA_SITFOLH AS Status, 
    dbo.SRA010.RA_TNOTRAB AS [Cod Tno], 
    dbo.SR6010.R6_DESC AS [Descrição Turno], 
    CONVERT(varchar, CONVERT(date, LEFT(dbo.SPC010.PC_DATA, 4) + '-' + SUBSTRING(dbo.SPC010.PC_DATA, 5, 2) + '-' + RIGHT(dbo.SPC010.PC_DATA, 2)), 103) AS Data, 
    dbo.SPC010.PC_PD AS [Cód Evento], 
    dbo.SP9010.P9_DESC AS [Decrição Evento], 
    dbo.SPC010.PC_ABONO AS Abono, 
    dbo.SPC010.PC_QUANTC AS [Horas Calculadas], 
    dbo.SPC010.PC_QTABONO AS [Horas Abonadas], 
    dbo.SPC010.PC_QUANTC - dbo.SPC010.PC_QTABONO AS [Horas Folha], 
    dbo.SRA010.RA_DEPTO AS [Cód Depto], 
    dbo.SQB010.QB_DESCRIC AS [Descrição Departamento] 
FROM 
    (((((dbo.SPC010 
    INNER JOIN dbo.SRA010 ON dbo.SPC010.PC_MAT = dbo.SRA010.RA_MAT) 
    INNER JOIN dbo.CTT010 ON dbo.SPC010.PC_FILIAL = dbo.CTT010.CTT_FILIAL AND dbo.SRA010.RA_CC = dbo.CTT010.CTT_CUSTO) 
    INNER JOIN dbo.SP9010 ON dbo.SPC010.PC_PD = dbo.SP9010.P9_CODIGO) 
    INNER JOIN dbo.SR6010 ON dbo.SRA010.RA_TNOTRAB = dbo.SR6010.R6_TURNO) 
    INNER JOIN dbo.SQB010 ON dbo.SRA010.RA_DEPTO = dbo.SQB010.QB_DEPTO) 
    INNER JOIN dbo.SRJ010 ON dbo.SRA010.RA_CODFUNC = dbo.SRJ010.RJ_FUNCAO 
WHERE 
    (dbo.SPC010.PC_QUANTC - dbo.SPC010.PC_QTABONO) > 0 
    --AND dbo.SPC010.PC_PD NOT IN ('027', '003', '110', '112', '011', '017', '007', '033', '009', '038', '123', '044', '048', '127', '060', '064', '126', '129', '068', '072', '128', '125', '052', '056', '124', '122', '050', '046', '062', '058', '070', '066', '054', '042', '078', '076', '084', '082', '088', '086', '080', '074', '098', '096', '104', '102', '108', '106', '100', '094', '013', '019') 
    AND dbo.SPC010.D_E_L_E_T_ <> '*' 
    AND dbo.CTT010.D_E_L_E_T_ <> '*' 
    AND dbo.SP9010.D_E_L_E_T_ <> '*' 
    AND dbo.SPC010.PC_PDI = '' 
    AND dbo.SR6010.D_E_L_E_T_ <> '*' 
    AND dbo.SQB010.D_E_L_E_T_ <> '*' 
    AND dbo.SRJ010.D_E_L_E_T_ <> '*' 
ORDER BY 
    dbo.SPC010.PC_MAT, dbo.SPC010.PC_DATA, dbo.SPC010.PC_PD;
"""
protheus = pd.read_sql_query(query, con_tot)